In [ ]:
from src.barycentre import *
from natsort import natsorted
import pandas as pd
import numpy as np
import glob
import math

In [ ]:
photos_paths = glob.glob("photos/*")
photos_paths = natsorted(photos_paths)

In [ ]:
df = pd.DataFrame(np.arange(4839).reshape((1613, 3)), columns=['filename', 'barycenter', 'surface'])
df["filename"] = df["filename"].astype(str)
df["barycenter"] = df["barycenter"].astype(str)
df["surface"] = df["surface"].astype(int)

In [ ]:
with open("descripteurs.txt", "w") as filout:
    for i, photo_path in enumerate(photos_paths):
        image = face_recognition.load_image_file(photo_path)
        image_name = photo_path[7:]
        face = get_ancrage(image)
        if face == False:
            continue
        df.iloc[i] = {'filename' : image_name, 'barycenter' : (face[0][0],face[0][1]), 'surface': face[1]}
        if i % 50 == 0:
            print(i)
        #filout.write(f"{image_name}|{face[0][0]},{face[0][1]},{face[1]}\n")

In [ ]:
x = df["filename"].str.contains("img") # selecting rows to supress, if not img

In [ ]:
df = df.drop(df[(x == False).values].index)

In [ ]:
df = df.sort_values(by="surface")

In [ ]:
df.to_json("dataframe.json")

In [ ]:
w = 2500 # to create bins of size 2500

n = math.ceil((df["surface"].max() - df["surface"].min())/w)

df.hist(column="surface", bins=n)

In [ ]:
df["surface"].describe()

In [ ]:
sample = df[(df["surface"] > 5600).values]

In [ ]:
# shuffleing rows
sample = sample.sample(frac=1).reset_index(drop=True)

In [ ]:
croissant = sample.iloc[:615].copy(deep=True)
croissant = croissant.sort_values(by="surface")

In [ ]:
decroissant = sample[615:].copy(deep=True)
decroissant = decroissant.sort_values(by="surface", ascending=False)

In [ ]:
croissant.drop([30, 308, 596], inplace = True) # dropping last big faces

In [ ]:
with open("croissant_decroissant.txt", "w") as filout:
    for row in croissant.itertuples():
        filout.write(f"{row[1]}|{tuple(row[2])[0]},{tuple(row[2])[1]}\n")
    for row in decroissant.itertuples():
        filout.write(f"{row[1]}|{tuple(row[2])[0]},{tuple(row[2])[1]}\n")